In [16]:
import PIL
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import keras

from keras import layers
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Conv1D, Flatten
from keras.layers import CuDNNLSTM, GRU, LSTM, ConvLSTM2D, CuDNNGRU
from keras.layers.normalization import BatchNormalization

from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import tensorflow

In [2]:
image = Image.open("circle.png")

In [3]:
print(image.size)
print(image.getpixel((1,1)))
print(image.getpixel((500,500)))

(913, 824)
(255, 255, 255)
(255, 127, 39)


In [4]:
# generate data

rows_list = list()
for i in range(10000):
    rand_x = np.random.randint(image.size[0])
    rand_y = np.random.randint(image.size[1])
    if image.getpixel((rand_x, rand_y)) == (255, 127, 39):  # ms orange lul
        rows_list.append({"x":rand_x, "y":rand_y, "color":1})
    else:
        rows_list.append({"x":rand_x, "y":rand_y, "color":0})

df = pd.DataFrame(rows_list)
df.head(20)

,color,x,y
0,0,841,636
1,1,341,237
2,0,578,737
3,0,296,678
4,0,861,165
5,1,375,321
6,0,873,669
7,0,42,717
8,0,867,31
9,0,210,781


In [5]:
features = df.drop(columns=["color"])
target = df["color"]
t_in, v_in, t_out, v_out = train_test_split(features, target, test_size = 0.2, random_state=0)

In [6]:
rf = RandomForestClassifier(n_estimators = 100, random_state=0, n_jobs=4, verbose=1)
rf.fit(t_in, t_out)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
                       oob_score=False, random_state=0, verbose=1,
                       warm_start=False)

In [7]:
def acc_printer(pred, actual):
    assert len(pred) == len(actual)
    fp=0;tp=0;fn=0;tn=0
    for i in range(len(pred)):
        if pred[i]==actual[i]==1:
            tp+=1
        elif pred[i]==actual[i]==0:
            tn+=1
        elif 1==pred[i]!=actual[i]:
            fp+=1
        elif 0==pred[i]!=actual[i]:
            fn+=1
        else:
            raise Exception("wtf")
    print("Stats:\nCorrect true: %s\nCorrect false: %s\nFalse positive: %s\nFalse negative: %s\n" % (tp, tn, fp, fn))

In [8]:
sols = rf.predict(v_in)
acc_printer(sols, v_out.values)

Stats:
Correct true: 367
Correct false: 1620
False positive: 8
False negative: 5



[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [17]:
def cnn_gru_model(data):
    #input layer
    input_x = keras.Input(shape=(np.shape(data['x_train'])[1], 1), name="x_input")
    print(input_x)
    
    conv = (Conv1D(64, 1, activation='relu'))(input_x)
    conv = BatchNormalization()(conv)
    conv = (Conv1D(64, 2, activation='relu'))(conv)
    conv = BatchNormalization()(conv)
    
    gru = (CuDNNGRU(64, return_sequences=True))(conv)
    gru = (CuDNNGRU(32, return_sequences=True))(gru)
    gru = (CuDNNGRU(16, return_sequences=False))(gru)
    
    gru = layers.Dense(64, activation='sigmoid')(gru)
    gru = layers.Dense(32, activation='sigmoid')(gru)
    y = layers.Dense(1, activation='sigmoid')(gru)
    
    #model
    #classifier = Model([input_x, input_xd], y)

    classifier = Model([input_x], y)
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

def train_model(model, t_in, v_in, t_out, v_out, MODEL_NAME, class_weight=None):
    callbacks = [EarlyStopping(monitor='val_loss', patience=PATIENCE),
             ModelCheckpoint(filepath=MODEL_PATH+"/"+MODEL_NAME+".h5", monitor='val_loss', verbose=2, save_best_only=True)]
    model.summary()
    model.fit(x={"x_input": t_in}, y=t_out, shuffle=True, epochs=EPOCHS, callbacks=callbacks, batch_size=BATCH_SIZE,
              verbose=1, validation_data=([v_in], v_out), class_weight=class_weight)
    return model

network = cnn_gru_model({'x_train': t_in})
model = train_model(network)


Tensor("x_input_4:0", shape=(None, 2, 1), dtype=float32)


ModuleNotFoundError: No module named 'tensorflow.contrib'

In [ ]:
sols = rf.predict(v_in)
acc_printer(sols, v_out.values)

In [20]:
keras.__version__

'2.3.1'